In [115]:
import pandas as pd
from datetime import datetime
import numpy as np
import time

# Upsert

In [80]:
import pandas as pd

def pandas_upsert_with_change_date(
    df_main: pd.DataFrame,
    df_update: pd.DataFrame,
    key: str,
    *,
    tracked_cols=None,
    change_date_col=None,
    change_date=None,
    allow_schema_evolution: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    main = df_main.copy()
    update = df_update.copy()


    # Defaults
    if change_date is None:
        change_date = pd.Timestamp.today().normalize()

    if tracked_cols is not None and isinstance(tracked_cols, str):
        tracked_cols = [tracked_cols]


    # Schema evolution
    added_columns = []
    if allow_schema_evolution:
        added_columns = list(set(update.columns) - set(main.columns))
        for col in added_columns:
            main[col] = pd.NA

        if change_date_col and change_date_col not in main.columns:
            main[change_date_col] = pd.NaT


    # Validation

    if key not in main.columns or key not in update.columns:
        raise ValueError(f"Key '{key}' must exist in both dataframes")

    # set index to chosen column
    main = main.set_index(key)
    update = update.set_index(key)

    common_cols = list(set(main.columns) & set(update.columns))
    intersect_idx = main.index.intersection(update.index)


    # Detect changes BEFORE update (FOR LOG)
    rows_with_any_change = set()
    rows_with_tracked_change = set()
    change_counts = {}

    for col in common_cols:
        old = main.loc[intersect_idx, col]
        new = update.loc[intersect_idx, col]

        mask = new.notna() & (old != new)
        changed_rows = intersect_idx[mask]

        change_counts[col] = int(mask.sum())
        rows_with_any_change.update(changed_rows)

        if tracked_cols and col in tracked_cols:
            rows_with_tracked_change.update(changed_rows)

    rows_updated = len(rows_with_any_change)

    # Apply update
    main.update(update[common_cols])

    # Apply change-date logic
    if tracked_cols and change_date_col:
        main.loc[list(rows_with_tracked_change), change_date_col] = change_date

    # Insert new rows
    new_rows = update.loc[~update.index.isin(main.index)]
    rows_inserted = len(new_rows)

    main = pd.concat([main, new_rows], axis=0)

    result = main.reset_index()

    # Logging
    if verbose:
        print("UPSERT SUMMARY")
        print("-" * 40)
        print(f"Rows updated        : {rows_updated}")
        print(f"Rows inserted       : {rows_inserted}")
        print(f"Columns added       : {len(added_columns)}")

        if added_columns:
            print(f"  Added columns     : {added_columns}")

        changed_cols = {k: v for k, v in change_counts.items() if v > 0}
        if changed_cols:
            print("Column changes:")
            for col, cnt in changed_cols.items():
                print(f"  {col:<15} : {cnt}")
        else:
            print("Column changes      : none")

        if tracked_cols and change_date_col:
            print(f"{change_date_col} updated for {len(rows_with_tracked_change)} rows")

        print("-" * 40)

    return result


# Main process

In [47]:
# # Latest Category
# category_main = pd.read_excel("checkpoint_files/tracking_with_updated_category_daily_activity.xlsx")

# # get only whatsapp and category
# donor_category = category_main[['Whatsapp','Kategori']]

In [48]:
# category_update = database_all[['Whatsapp','Kategori','CRM']].merge(donor_category,how='left', on='Whatsapp')

In [49]:
# def update_category(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Kategori_y']):
#         return df['Kategori_x']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['Kategori_x'] != df['Kategori_y']:
#         return df['Kategori_y']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['Kategori_x']

# category_update['Kategori'] = category_update.apply(update_category, axis=1)

# Start Here

In [91]:
pd.set_option('display.max_rows', None)

# Current database
database_all = pd.read_excel("Database.xlsx")

# Latest category
category_update = pd.read_excel("checkpoint_files/category_each_donor.xlsx")

In [92]:
category_update = category_update[['Whatsapp', 'kategori_update', 'terakhir_donasi', 'terakhir_respon',
       'terakhir_action', 'kategori_prioritas', 'terakhir_interaksi',
       'start_date', 'CRM', 'kategori_karantina', 'Tanggal_terakhir_donasi',
       'tanggal_karantina', 'cleansing_ke-', 'priority_updated_at',
       'cleansing_updated_at']]

In [93]:
# function for updating Tanggal_terakhir_donasi
updated_category = pandas_upsert_with_change_date(
    category_update,
    database_all[['Whatsapp','CRM','Tanggal_terakhir_donasi']],
    key="Whatsapp"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 3049
Rows inserted       : 1641
Columns added       : 0
Column changes:
  Tanggal_terakhir_donasi : 1148
  CRM             : 1934
----------------------------------------


In [84]:
# for setting a start date based on time when data is assigned to CRM
crm_start = {
    # "WA_7778": pd.Timestamp("2025-09-01"),
    # "WA_0056": pd.Timestamp("2025-10-01"),
    # "WA_0036": pd.Timestamp("2025-10-01"),
    # "WA_2056": pd.Timestamp("2025-10-01"),
    # "WA_2057": pd.Timestamp("2025-10-01"),
    # "WA_1097": pd.Timestamp("2025-10-01"),
    # "WA_0041": pd.Timestamp("2025-10-10"),
    # "WA_0051": pd.Timestamp("2025-10-10"),
    # "WA_0046": pd.Timestamp("2025-10-15"),
    # "WA_0047": pd.Timestamp("2025-10-15"),
    # "WA_1095": pd.Timestamp("2025-10-20"),
    # "WA_1096": pd.Timestamp("2025-10-20"),
    # "WA_2058": pd.Timestamp("2025-10-17"),
    # "WA_2059": pd.Timestamp("2025-10-17"),
    # "WA_0031": pd.Timestamp("2025-10-25"),
    # "WA_0032": pd.Timestamp("2025-10-25"),
    # "WA_0034": pd.Timestamp("2025-10-25"),
    # "WA_8196": pd.Timestamp("2025-10-25"),
    # "WA_1094": pd.Timestamp("2025-10-25"),
    # "WA_8194": pd.Timestamp("2025-10-31"),
    # "WA_8195": pd.Timestamp("2025-10-31"),
    # "Refill_WA_1097_2nd": pd.Timestamp("2025-12-15"),
    # "refill_WA_0041": pd.Timestamp("2025-11-04"),
    # "refill_WA_0051": pd.Timestamp("2025-11-04"),
    # "Refill_WA_2056": pd.Timestamp("2025-10-26"),
    # "Refill_WA_2057": pd.Timestamp("2025-10-26"),
    # "Refill_WA_0046": pd.Timestamp("2025-11-04"),
    # "Refill_WA_0047": pd.Timestamp("2025-11-04"),
    # "Refill_WA_1095": pd.Timestamp("2025-11-10"),
    # "Refill_WA_1096": pd.Timestamp("2025-11-10"),
    # "Refill_WA_2058": pd.Timestamp("2025-11-10"),
    # "Refill_WA_2059": pd.Timestamp("2025-11-10"),
    # "refill_WA_0031": pd.Timestamp("2025-11-17"),
    # "refill_WA_0032": pd.Timestamp("2025-11-17"),
    # "Refill_WA_8196": pd.Timestamp("2025-12-10"),
    # "Refill_WA_0034": pd.Timestamp("2025-11-25"),
    # "refill_WA_0041_2": pd.Timestamp("2025-12-10"),
    # "Refill_WA_0046_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_0034_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_0047_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_0056_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_1095_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_2056_2nd": pd.Timestamp("2025-12-20"),
    # "Refill_WA_2057_2nd": pd.Timestamp("2025-12-20"),
    # "refill_WA_0051_2": pd.Timestamp("2025-12-29"),
    # "Refill_WA_2059_2nd": pd.Timestamp("2025-12-15"),
    # "Refill_WA_1096_2nd": pd.Timestamp("2025-12-30"),
    # "WA_8271": pd.Timestamp("2025-12-03"),
    # "WA_8197": pd.Timestamp("2025-12-03"),
    # "WA_8204": pd.Timestamp("2025-12-03"),
    # "WA_8272": pd.Timestamp("2025-12-03"),
    # "WA_8203": pd.Timestamp("2025-12-03"),
    # "WA_1092": pd.Timestamp("2025-12-03"),
    # "Refill_WA_7778_2": pd.Timestamp("2025-12-31"),
    # "Refill_WA_8194": pd.Timestamp("2026-01-10"),
    # "Refill_WA_8195": pd.Timestamp("2026-01-10"),
    # "WA_8173": pd.Timestamp("2026-01-10"),
    # "WA_8172": pd.Timestamp("2026-01-10"),
    # "Refill_WA_2058_2": pd.Timestamp("2026-01-26")
    
}

# mapped = updated_category['CRM'].map(crm_start)

# updated_category['start_date'] = (
#     updated_category['start_date']
#     .where(mapped.isna(), mapped)
# )

In [85]:
# crm_cleanse = {
#     "WA_7778": 4,
#     "WA_0056": 2,
#     "WA_0036": 2,
#     "WA_2056": 2,
#     "WA_2057": 2,
#     "WA_1097": 2,
#     "WA_0041": 2,
#     "WA_0051": 2,
#     "WA_0046": 2,
#     "WA_0047": 2,
#     "WA_1095": 2,
#     "WA_1096": 2,
#     "WA_2058": 2,
#     "WA_2059": 2,
#     "WA_0031": 2,
#     "WA_0032": 2,
#     "WA_0034": 2,
#     "WA_8196": 2,
#     "WA_1094": 2,
#     "WA_8194": 1,
#     "WA_8195": 1,
#     "Refill_WA_1097_2nd": 0,
#     "refill_WA_0041": 1,
#     "refill_WA_0051": 1,
#     "Refill_WA_2056": 1,
#     "Refill_WA_2057": 1,
#     "Refill_WA_0046": 1,
#     "Refill_WA_0047": 1,
#     "Refill_WA_1095": 1,
#     "Refill_WA_1096": 1,
#     "Refill_WA_2058": 1,
#     "Refill_WA_2059": 1,
#     "refill_WA_0031": 1,
#     "refill_WA_0032": 1,
#     "Refill_WA_8196": 0,
#     "Refill_WA_0034": 0,
#     "refill_WA_0041_2": 0,
#     "Refill_WA_0046_2nd": 0,
#     "Refill_WA_0034_2nd": 0,
#     "Refill_WA_0047_2nd": 0,
#     "Refill_WA_0056_2nd": 0,
#     "Refill_WA_1095_2nd": 0,
#     "Refill_WA_2056_2nd": 0,
#     "Refill_WA_2057_2nd": 0,
#     "refill_WA_0051_2": 0,
#     "Refill_WA_2059_2nd": 0,
#     "Refill_WA_1096_2nd": 0,
#     "WA_8271": 0,
#     "WA_8197": 0,
#     "WA_8204": 0,
#     "WA_8272": 0,
#     "WA_8203": 0,
#     "WA_1092": 0,
#     "Refill_WA_7778_2": 0,
#     "Refill_WA_8194": 0,
#     "Refill_WA_8195": 0,
#     "WA_8173": 0,
#     "WA_8172": 0,
# }


# updated_category['cleansing_ke-'] = (
#     updated_category['CRM']
#     .map(crm_cleanse)
#     .fillna(updated_category['cleansing_ke-'])
#     .astype(int)
# )

In [94]:
# if start_date less than Tanggal_terakhir_donasi, update start_date to Tanggal_terakhir_donasi
mask = (
    updated_category["start_date"].notna() &
    updated_category["Tanggal_terakhir_donasi"].notna() &
    (updated_category["start_date"] < updated_category["Tanggal_terakhir_donasi"])
)

updated_category.loc[mask, "start_date"] = updated_category.loc[mask, "Tanggal_terakhir_donasi"]

In [95]:
# ----------------------------------------------- #
    # FILL HERE IF THERE IS A DATA CLEANSING #
# ----------------------------------------------- #

# daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1097.xlsx")
# daily_2056_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (BONDING).xlsx")
# daily_2056_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (MAINTAIN).xlsx")
# daily_2057_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (BONDING).xlsx")
# daily_2057_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (MAINTAIN).xlsx")
# daily_0046 = pd.read_excel("../data_blast/daily_activity/filled/new/0046_DA_Shania.xlsx")
# daily_0047 = pd.read_excel("../data_blast/daily_activity/filled/new/0047_DA_Shania.xlsx")
# daily_0041 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0041.xlsx")
# daily_0051 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0051.xlsx")
# daily_2031 = pd.read_excel("../data_blast/daily_activity/filled/old/Daily_Activity_WA_2031.xlsx")
# daily_2032 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx")
# daily_7778 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_7778.xlsx")

# November
# daily_0034 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0034.xlsx")
# daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name='Data Bonding')

# Desember
# daily_0046 = pd.read_excel("../data_blast/daily_activity/filled/new/0046_DA_Shania_update.xlsx", sheet_name="Donatur Bonding ")
# daily_0047 = pd.read_excel("../data_blast/daily_activity/filled/new/0047_DA_Shania_update.xlsx", sheet_name="Data Bonding")
# daily_2031 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_WA_2031.xlsx", sheet_name="Donatur Baru Bonding ")
# daily_2032 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx", sheet_name="Donatur Baru Bonding")
# daily_8196 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_8196.xlsx", sheet_name="Donatur Baru ")
# daily_0041_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0041.xlsx", sheet_name="Donatur Bonding")
# daily_1097_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name="Data Maintenance")
# daily_1097_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name="Data Bonding")
# daily_2059_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_2059.xlsx", sheet_name="Donatur Baru")
# daily_2059_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_2059.xlsx", sheet_name="Donatur Lama")
# daily_0051 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0051.xlsx", sheet_name="Donatur Bonding")
# daily_0034 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0034.xlsx", sheet_name="Data Bonding")
# daily_1095_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1095.xlsx", sheet_name="Donatur Baru")
# daily_1095_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1095.xlsx", sheet_name="Donatur Lama")
# daily_7778 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_7778.xlsx", sheet_name="DATA DONATUR BARU 7778")
# daily_1096_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1096.xlsx", sheet_name="Donatur Baru")
# daily_8195 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_8195.xlsx", sheet_name="Donatur Baru")
# daily_8194 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_8194.xlsx", sheet_name="Donatur Baru")

# January
# daily_0034_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0034.xlsx", sheet_name="Data Maintenance")
# daily_2056_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_2056.xlsx", sheet_name="New Interaksi Lanjutan 2")
# daily_2057_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_2057.xlsx", sheet_name="New Interaksi Lanjutan 2")
# daily_0034_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0034.xlsx", sheet_name="Data Bonding baru")
# daily_2058_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_2058.xlsx", sheet_name="Donatur Baru ")
# daily_1095_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1095.xlsx", sheet_name="DATA BARU")
daily_2032_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx", sheet_name="Donatur Baru Bonding")
daily_2031_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_WA_2031.xlsx", sheet_name="Donatur Baru Bonding ")


# daily_2056 = pd.concat([daily_2056_bonding, daily_2056_maintain])
# daily_2057 = pd.concat([daily_2057_bonding, daily_2057_maintain])


daily_activity_all = pd.concat([daily_2031_bonding, daily_2032_bonding])
daily_activity_all = daily_activity_all.dropna(subset=['Whatsapp'])

In [96]:
# rename Tanggal 6 to Tanggal 3
daily_activity_all['Tanggal 3'] = daily_activity_all['Tanggal 3'].fillna(daily_activity_all['Tanggal 2']).fillna(daily_activity_all['Tanggal 1'])

category_updater = updated_category.merge(daily_activity_all[['Whatsapp','Next Action', 'Tanggal 3','Action 3', 'Respon 3', 'Donasi 3']], how='left', on='Whatsapp')

# standarize the value, for date and prioritas
category_updater['Tanggal 3'] = pd.to_datetime(category_updater['Tanggal 3'], errors='coerce')
category_updater.loc[category_updater['Next Action'] == "Prioirtas 3", "Next Action"] = "Prioritas 3"

# fill missing kategori_prioritas and terakhir_interaksi from daily activity
category_updater['kategori_prioritas'] = category_updater['kategori_prioritas'].fillna(category_updater['Next Action'])
category_updater['terakhir_interaksi'] = category_updater['terakhir_interaksi'].fillna(category_updater['Tanggal 3'])
category_updater['terakhir_donasi'] = category_updater['terakhir_donasi'].fillna(category_updater['Donasi 3'])
category_updater['terakhir_action'] = category_updater['terakhir_action'].fillna(category_updater['Action 3'])
category_updater['terakhir_respon'] = category_updater['terakhir_respon'].fillna(category_updater['Respon 3'])

# update category when on the daily activity is eliminasi or invalid then return that value
category_updater['kategori_update'] = np.where(category_updater['kategori_prioritas'].isin(["Eliminasi", "Invalid"]), category_updater['kategori_prioritas'], category_updater['kategori_update'])

In [97]:
daily_activity_all[daily_activity_all['Whatsapp']==6282311899895]

,l,Donatur,Whatsapp,Donasi Sebelumnya,Periode Prospek,Program,Entri Donatur,Tanggal 1,Sesi Blast,Konten,...,Donasi 3,Catatan,Next Action,No,Sesi Blast 1,Konten Blasting,Sesi Blast 2,Konten Blasting.1,Sesi Blast 3,Konten Blasting.2
26,27.0,Anonim,6282311899895,NaN,Masa Gajian,SUMATERA,December,2025-12-13,Sesi 1,Inisiasi Sumatera,...,NaN,po,Invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
category_updater = category_updater[['Whatsapp', 'kategori_update', 'terakhir_donasi', 'terakhir_respon',
       'terakhir_action', 'kategori_prioritas', 'terakhir_interaksi',
       'start_date', 'CRM', 'kategori_karantina', 'Tanggal_terakhir_donasi',
       'tanggal_karantina', 'cleansing_ke-', 'priority_updated_at',
       'cleansing_updated_at', 'Next Action']]

In [99]:
data_with_updated_previous_interactions = pandas_upsert_with_change_date(
    updated_category,
    category_updater,
    key="Whatsapp"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 869
Rows inserted       : 0
Columns added       : 1
  Added columns     : ['Next Action']
Column changes:
  kategori_update : 110
  terakhir_donasi : 2
  terakhir_action : 869
  terakhir_respon : 869
  kategori_prioritas : 869
  terakhir_interaksi : 869
  Next Action     : 869
----------------------------------------


In [100]:
def update_cleansing_monotonic(
    df_old: pd.DataFrame,
    df_new: pd.DataFrame,
    key: str = "Whatsapp",
    verbose: bool = True
) -> pd.DataFrame:

    old = df_old.set_index(key).copy()
    new = df_new.set_index(key).copy()

    old["cleansing_ke-"] = old["cleansing_ke-"].fillna(0)

    intersect_idx = old.index.intersection(new.index)

    old_date = old.loc[intersect_idx, "terakhir_interaksi"]
    new_date = new.loc[intersect_idx, "terakhir_interaksi"]

    mask_new_daily = (
        new_date.notna() &
        (old_date.isna() | (new_date > old_date))
    )

    idx_update = mask_new_daily.index[mask_new_daily]

    # ---- snapshot BEFORE update (for logging)
    before = old.loc[idx_update, ["cleansing_ke-"]].rename(
        columns={"cleansing_ke-": "cleansing_before"}
    )

    # ---- increment (NO RESET)
    old.loc[idx_update, "cleansing_ke-"] += 1
    old.loc[idx_update, "terakhir_interaksi"] = new_date.loc[idx_update]
    old.loc[idx_update, "cleansing_updated_at"] = pd.Timestamp.today().normalize()

    # ---- snapshot AFTER update
    after = old.loc[idx_update, ["cleansing_ke-"]].rename(
        columns={"cleansing_ke-": "cleansing_after"}
    )

    result = old.reset_index()

    if verbose:
    # build transition log
        transition_log = (
                old.loc[idx_update, ["CRM", "cleansing_ke-"]]
                .rename(columns={"cleansing_ke-": "cleansing_after"})
                .join(
                        before.reset_index().set_index(key)["cleansing_before"],
                        on=key
                )
                .reset_index()
        )

        summary = (
                transition_log
                .groupby(
                        ["CRM", "cleansing_before", "cleansing_after"]
                )
                .size()
                .rename("rows_changed")
                .reset_index()
                .sort_values(["CRM", "cleansing_before"])
        )

        print("CLEANSING MONOTONIC UPDATE")
        print("-" * 60)
        print(f"Rows checked         : {len(intersect_idx)}")
        print(f"Rows incremented : {len(idx_update)}")
        print("\nTransition summary per CRM:")
        print(summary.to_string(index=False))
        print("-" * 60)

    return result


In [101]:
data_after_cleansing = update_cleansing_monotonic(
    df_old=data_with_updated_previous_interactions, 
    df_new=category_updater)

CLEANSING MONOTONIC UPDATE
------------------------------------------------------------
Rows checked         : 89952
Rows incremented : 0

Transition summary per CRM:
Empty DataFrame
Columns: [CRM, cleansing_before, cleansing_after, rows_changed]
Index: []
------------------------------------------------------------


In [102]:
# def category_update(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Next Action']):
#         return df['kategori_prioritas']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['kategori_prioritas'] != df['Next Action']:
#         return df['Next Action']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['kategori_prioritas']

In [103]:
data_with_updated_interaksi_terakhir = pandas_upsert_with_change_date(
    data_after_cleansing,
    category_updater,
    key="Whatsapp"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 0
Rows inserted       : 0
Columns added       : 0
Column changes      : none
----------------------------------------


In [104]:
# FOR UPDATE THE DATE WHEN PRIORITY IS CHANGED
import pandas as pd
from collections import Counter

def update_priority_with_date(
    df_old: pd.DataFrame,
    df_new: pd.DataFrame,
    key: str = "Whatsapp",
    verbose: bool = True
) -> pd.DataFrame:

    old = df_old.set_index(key).copy()
    new = df_new.set_index(key).copy()

    # Align indexes
    intersect_idx = old.index.intersection(new.index)

    # Detect priority change
    old_priority = old.loc[intersect_idx, "kategori_prioritas"]
    new_priority = new.loc[intersect_idx, "kategori_prioritas"]

    mask_changed = (
        new_priority.notna() &
        (old_priority != new_priority)
    )


    # Build transition log
    transitions = Counter()

    for o, n in zip(old_priority[mask_changed], new_priority[mask_changed]):
        transitions[f"{o} → {n}"] += 1


    # Apply update
    old.update(new[["kategori_prioritas"]])

    today = pd.Timestamp.today().normalize()
    old.loc[mask_changed.index[mask_changed], "priority_updated_at"] = today

    result = old.reset_index()


    # Logging
    if verbose:
        print("PRIORITY UPSERT SUMMARY")
        print("-" * 40)
        print(f"Total rows checked   : {len(intersect_idx)}")
        print(f"Priority changes     : {int(mask_changed.sum())}")

        if transitions:
            print("Priority transitions:")
            for k, v in transitions.items():
                print(f"  {k:<20} : {v}")
        else:
            print("Priority transitions: none")

        print("-" * 40)

    return result

In [105]:
data_with_updated_priority = update_priority_with_date(
    df_old=data_with_updated_interaksi_terakhir, 
    df_new=category_updater
)

PRIORITY UPSERT SUMMARY
----------------------------------------
Total rows checked   : 89952
Priority changes     : 0
Priority transitions: none
----------------------------------------


In [106]:
# data_with_updated_interaksi_terakhir = pandas_upsert_with_change_date(
#     updated_category,
#     data_with_updated_priority,
#     key="Whatsapp"
# )

In [107]:
data_with_updated_priority.head()

,Whatsapp,kategori_update,terakhir_donasi,terakhir_respon,terakhir_action,kategori_prioritas,terakhir_interaksi,start_date,CRM,kategori_karantina,Tanggal_terakhir_donasi,tanggal_karantina,cleansing_ke-,priority_updated_at,cleansing_updated_at,Next Action
0,62881082355438,Aktif,NaN,NaN,NaN,NaN,NaT,2025-02-25 01:23:58,WA_2060,Aktif,2025-02-25 01:23:58,NaT,0.0,NaN,NaT,<NA>
1,6285753301575,Aktif,NaN,NaN,NaN,NaN,NaT,2025-02-28 06:47:06,WA_2060,Aktif,2025-02-28 06:47:06,NaT,0.0,NaN,NaT,<NA>
2,6282236899125,Aktif,NaN,NaN,NaN,NaN,NaT,2025-02-27 21:37:03,WA_2060,Aktif,2025-02-27 21:37:03,NaT,0.0,NaN,NaT,<NA>
3,628226357000,Invalid,NaN,NaN,NaN,NaN,NaT,2025-02-24 17:24:59,Eliminated,Aktif,2025-02-24 17:24:59,NaT,0.0,NaN,NaT,<NA>
4,628561889392,Loyal,NaN,NaN,NaN,NaN,NaT,2025-12-07 14:55:55,WA_2060,Aktif,2025-08-26 04:20:28,NaT,0.0,NaN,NaT,<NA>


In [108]:
data_with_updated_interaksi_terakhir[data_with_updated_interaksi_terakhir['CRM']=="refill_WA_0031"]['cleansing_ke-'].value_counts()

cleansing_ke-
1.0    641
Name: count, dtype: int64

In [ ]:
from pathlib import Path


def apply_lifecycle_rules(
    df: pd.DataFrame,
    today=None,
    verbose: bool = True,
    log_changes: bool = True,
    log_path: str | None = "lifecycle_changes.csv",
) -> pd.DataFrame:
    df = df.copy()

    if today is None:
        today = pd.Timestamp.today().normalize()


    # Ensure required columns

    df["kategori_karantina"] = df["kategori_karantina"].fillna("Aktif")

    if "tanggal_karantina" not in df.columns:
        df["tanggal_karantina"] = pd.NaT

    if "Whatsapp" not in df.columns:
        raise ValueError("Column 'Whatsapp' is required for lifecycle logging")

    # Snapshot before changes
    prev_status = df["kategori_karantina"].copy()


    # 1. RESET start_date ON ANY NEW DATE IN NECESSARY COLUMNS

    activation_cols = [
        c for c in
["start_date", "Tanggal_terakhir_donasi", "priority_update_at"]
        if c in df.columns
    ]

    df["start_date"] = df[activation_cols].max(axis=1)


    # 2. TURN PASSIVE

    # 1. Direct passive: Invalid / Eliminasi
    mask_direct_passive = (
        (df["kategori_karantina"] == "Aktif") &
        (df["kategori_prioritas"].isin(["Invalid", "Eliminasi"]))
    )

    # 2. Conditional passive: Prioritas 3 only
    mask_conditional_passive = (
        (df["kategori_karantina"] == "Aktif") &
        (df["kategori_prioritas"] == "Prioritas 3") &
        (today >= (df["start_date"] + pd.DateOffset(months=3))) &
        (df["cleansing_ke-"] % 3 == 0)
    )

    # 3. Combined rule
    mask_to_passive = mask_direct_passive | mask_conditional_passive

    df.loc[mask_to_passive, "kategori_karantina"] = "Pasif"
    df.loc[
        mask_to_passive & df["tanggal_karantina"].isna(),
        "tanggal_karantina"
    ] = today


    # 3. REACTIVATE
    mask_reactivate = (
        (df["kategori_karantina"] == "Pasif") &
        (~df["kategori_prioritas"].isin(["Invalid", "Eliminasi"])) &
        (
                # 1. Donasi setelah / pada interaksi
                (df["Tanggal_terakhir_donasi"] >= df["terakhir_interaksi"])

                |

                # 2. Donasi setelah masuk karantina
                (df["Tanggal_terakhir_donasi"] > df["tanggal_karantina"])

                |

                # 3. Donasi sedikit sebelum interaksi + interaksi recent
                (
                        df["Tanggal_terakhir_donasi"].between(
                                df["terakhir_interaksi"] - pd.DateOffset(months=1),
                                df["terakhir_interaksi"]
                        )
                        &
                        (df["terakhir_interaksi"] >= today - pd.DateOffset(months=2))
                )
        )
    )

    df.loc[mask_reactivate, "kategori_karantina"] = "Aktif"
    df.loc[mask_reactivate, "tanggal_karantina"] = pd.NaT


    # 4. CHANGE DETECTION (FOR LOG)

    new_status = df["kategori_karantina"]

    mask_aktif_to_pasif = (prev_status == "Aktif") & (new_status == "Pasif")
    mask_pasif_to_aktif = (prev_status == "Pasif") & (new_status == "Aktif")

    changes_df = pd.concat(
        [
                df.loc[mask_aktif_to_pasif, [
                        "Whatsapp", "CRM", "start_date", "tanggal_karantina"
                ]].assign(
                        status_before="Aktif",
                        status_after="Pasif",
                        change_date=today
                ),

                df.loc[mask_pasif_to_aktif, [
                        "Whatsapp", "CRM", "start_date"
                ]].assign(
                        status_before="Pasif",
                        status_after="Aktif",
                        change_date=today,
                        tanggal_karantina=pd.NaT
                )
        ],
        ignore_index=True
    )


        # 5. LOGGING

    if verbose:
        print("Lifecycle update summary:")
        print(f"- Aktif → Pasif : {mask_aktif_to_pasif.sum()}")
        print(f"- Pasif → Aktif : {mask_pasif_to_aktif.sum()}")

        if not changes_df.empty:
                crm_summary = (
                        changes_df
                        .groupby(["CRM", "status_before", "status_after"])
                        .size()
                        .rename("rows_changed")
                        .reset_index()
                        .sort_values(["CRM", "status_before"])
                )

                print("\nChanges per CRM:")
                print(crm_summary.to_string(index=False))

                print("\nDetail changes (sample):")
                print(changes_df.head(10).to_string(index=False))

    # 6. EXPORT LOG FILE

    if log_changes and log_path and not changes_df.empty:
        log_path = Path(log_path)
        changes_df.to_csv(log_path, index=False)

        if verbose:
            print(f"\nLifecycle change log written to: {log_path.resolve()}")

    return df

In [110]:
data_with_updated_karantina = apply_lifecycle_rules(
    df=data_with_updated_priority)

Lifecycle update summary:
- Aktif → Pasif : 111
- Pasif → Aktif : 45

Changes per CRM:
           CRM status_before status_after  rows_changed
    Eliminated         Pasif        Aktif             3
Refill_WA_1097         Aktif        Pasif             2
Refill_WA_1097         Pasif        Aktif             2
       WA_0041         Pasif        Aktif             2
       WA_0046         Pasif        Aktif             3
       WA_0047         Pasif        Aktif             4
       WA_0051         Pasif        Aktif             4
       WA_1095         Pasif        Aktif             4
       WA_1097         Pasif        Aktif             5
       WA_2056         Aktif        Pasif             1
       WA_2056         Pasif        Aktif             5
       WA_2057         Aktif        Pasif             1
       WA_2057         Pasif        Aktif             3
       WA_2059         Pasif        Aktif             3
       WA_7778         Pasif        Aktif             7
refill_WA_0031   

PermissionError: [Errno 13] Permission denied: 'lifecycle_changes.csv'

In [111]:
pd.set_option("display.max_rows", None)
data_with_updated_karantina['CRM'].value_counts()

CRM
WA_2060               10673
WA_7778                2961
WA_0036                2504
WA_0056                2003
WA_2060_1              2000
Refill_WA_7778_2       1894
WA_8203                1706
WA_8271                1703
WA_1092                1699
WA_8272                1697
WA_8204                1696
WA_8197                1690
Eliminated             1477
WA_2057                1469
WA_2056                1431
WA_2056_1              1196
WA_2060_2               920
WA_1095                 891
WA_0047                 873
WA_0051                 871
WA_0046                 870
WA_0041                 869
WA_8194                 868
WA_1094                 868
WA_8195                 867
WA_0032                 866
WA_0034                 864
WA_1096                 864
WA_2059                 863
WA_0031                 863
WA_8173                 863
WA_2058                 863
WA_8172                 863
Refill_WA_1097          780
Refill_WA_1095          724
WA_1097         

In [112]:
final_result = pandas_upsert_with_change_date(
    category_update,
    data_with_updated_karantina,
    key="Whatsapp"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 3913
Rows inserted       : 1641
Columns added       : 1
  Added columns     : ['Next Action']
Column changes:
  kategori_update : 110
  CRM             : 1934
  terakhir_donasi : 2
  tanggal_karantina : 111
  terakhir_action : 869
  kategori_karantina : 156
  Tanggal_terakhir_donasi : 1148
  terakhir_respon : 869
  start_date      : 1110
  kategori_prioritas : 869
  terakhir_interaksi : 869
  Next Action     : 869
----------------------------------------


In [113]:
data_with_updated_karantina[data_with_updated_karantina['kategori_karantina'] == 'Pasif']['CRM'].value_counts()

CRM
WA_7778           2321
WA_2057           1122
WA_2056           1091
WA_0041            706
WA_2059            685
WA_0051            674
WA_0047            654
WA_0046            653
Refill_WA_1097     625
WA_1095            621
WA_1097            546
Eliminated         398
refill_WA_0031      57
Refill_WA_1095      51
refill_WA_0032      50
Refill_WA_0047      41
refill_WA_0051      35
refill_WA_0041      35
WA_0032             32
Refill_WA_0046      31
WA_1094             31
WA_0031             12
Refill_WA_2059       3
Name: count, dtype: int64

In [30]:
# update_category = update_category.drop_duplicates(subset=['Whatsapp'], keep='last')


In [114]:
data_with_updated_karantina.to_excel("checkpoint_files/category_each_donor.xlsx", index=False)